In [7]:
import gc

import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.layers import Dropout, Conv2D, MaxPooling2D, Input, Flatten, Dense
from keras.layers import MaxPooling1D, Conv1D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils.np_utils import to_categorical
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

# USE MIXED PRECISION 
MIX = True
if MIX:
    tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
    print("Mixed precision enabled")
else:
    print("Using full precision")

Mixed precision enabled


In [2]:
FS = 400
s, e = 15, 25
a, b = 0, 246
data = np.load("../data/processed/train_img.npy", allow_pickle=True).item()

fmri = data["fMRI"]
rsp = data["RSP"]
ppg = data["PPG"]
bio = np.concatenate((rsp, ppg), axis=-1)

# rsp = data["RSP"][:, 5*FS:]
# ppg = data["PPG"][:, 5*FS:]
# bio = np.stack((rsp[:, ::10], ppg[:, ::10]), axis=-1)
train = np.swapaxes(fmri[:, a:b, s:e], 1, 2)
# train = np.expand_dims(np.swapaxes(fmri[:, :, :], 1, 2),  axis=-1)

subject = data["subject"]
target = data["class"].astype(int) + 1
level = data["level"]

print(f"Data shape: {train.shape}")
print(f"Bio shape: {bio.shape}")
print(f"Subject shape: {subject.shape}")
print(f"Target shape: {target.shape}")
print(f"Level shape: {level.shape}")
print(np.unique(target))

Data shape: (480, 10, 246)
Bio shape: (480, 64, 64, 4)
Subject shape: (480,)
Target shape: (480,)
Level shape: (480,)
[0 1 2]


In [3]:
data_test = np.load("../data/processed/test_img.npy", allow_pickle=True).item()
test_idx = np.where(~np.isnan(data_test["class"]))[0]

y_test = data_test["class"][test_idx].astype(int) + 1
y_test_level = data_test["level"][test_idx]
fmri_test = np.swapaxes(data_test["fMRI"][:, a:b, s:e], 1, 2)
bio_test = np.concatenate((data_test["RSP"][:], data_test["PPG"][:]), axis=-1)

print(f"fMRI shape: {fmri_test.shape}")
print(f"Bio shape: {bio_test.shape}")
print(f"Test target shape: {y_test.shape}")
print("Level shape:", y_test_level.shape)
print(np.unique(y_test))

fMRI shape: (120, 10, 246)
Bio shape: (120, 64, 64, 4)
Test target shape: (36,)
Level shape: (36,)
[0 1 2]


In [4]:
def build_model():
    # Define input shapes
    input_shape_1d = (train.shape[1], train.shape[2])  # Temporal input: 25 time points for 246 brain regions
    input_shape_2d = (train.shape[1], train.shape[2], 1)  # Spatial input: 25x246 spatial grid
    l2_reg = 1e-2
    dp = 0.25

    # 1D CNN for temporal data
    input_1d = Input(shape=input_shape_1d)
    # x1 = TimeDistributed(Dense(128, kernel_regularizer=l2(l2_reg), activation="relu"))(input_1d)
    # x1 = LSTM(64, return_sequences=False, kernel_regularizer=l2(l2_reg))(input_1d)
    # x1 = Dropout(dp)(x1)

    x1 = Conv1D(filters=32, kernel_size=3, padding="valid", activation="relu", kernel_regularizer=l2(l2_reg))(input_1d)
    x1 = Conv1D(filters=64, kernel_size=5, padding="valid", activation="relu", kernel_regularizer=l2(l2_reg))(x1)
    x1 = MaxPooling1D(pool_size=2)(x1)
    x1 = Dropout(dp)(x1)
    x1 = Flatten()(x1)

    # 2D CNN for spatial data
    input_2d = Input(shape=input_shape_2d)
    x2 = Conv2D(filters=64, kernel_size=3, padding="valid", activation="relu", kernel_regularizer=l2(l2_reg))(input_2d)
    x2 = Conv2D(filters=128, kernel_size=5, padding="valid", activation="relu", kernel_regularizer=l2(l2_reg))(x2)
    x2 = MaxPooling2D(pool_size=2)(x2)
    x2 = Dropout(dp)(x2)
    x2 = Flatten()(x2)

    # Concatenate the outputs of the two branches
    combined = concatenate([x1, x2])

    # Fully connected layers
    fc = Dense(256, activation="relu", kernel_regularizer=l2(l2_reg))(combined)
    fc = Dropout(dp)(fc)
    fc = Dense(128, activation="relu", kernel_regularizer=l2(l2_reg))(fc)

    # Output layer
    class_output = Dense(3, activation="softmax", name="class")(fc)
    level_output = Dense(1, activation="linear", name="level")(fc)

    model = Model(inputs=[input_1d, input_2d], outputs=[class_output, level_output])

    # optimizer = RMSprop(learning_rate=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer = Adam(learning_rate=1e-4)
    # optimizer = SGD(learning_rate=0.1, momentum=0.9, nesterov=True)
    model.compile(optimizer=optimizer, loss={"class": "categorical_crossentropy", "level": "mae"},
                  metrics={"class": "accuracy", "level": "mae"})

    return model

In [9]:
epochs = 50
batch_size = 32
val_acc = []
test_count = []
test_score = []
preds = []
rocauc_test = []
test_acc = []
val_mae = []
val_smape = []
test_smape = []
accs = []

X = train
y = target
groups = subject

loso_tidx = np.load("../data/loso_tidx.npy", allow_pickle=True)
loso_vidx = np.load("../data/loso_vidx.npy", allow_pickle=True)

cv = LeaveOneGroupOut()
# cv = GroupKFold(n_splits=5)
# cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
for i, (tidx, vidx) in enumerate(cv.split(X, y, groups)):
    print("#" * 50)
    print(f"### Fold {i + 1}")

    # X_train, X_val = X[tidx], X[vidx]
    # y_train, y_val = to_categorical(y[tidx]), to_categorical(y[vidx])
    # y_train_level, y_val_level = level[tidx], level[vidx]
    
    X_train, X_val = X[loso_tidx[i]], X[loso_vidx[i]]
    y_train, y_val = to_categorical(y[loso_tidx[i]]), to_categorical(y[loso_vidx[i]])
    y_train_level, y_val_level = level[loso_tidx[i]], level[loso_vidx[i]]

    print(f"### train size {len(tidx)}, valid size {len(vidx)}")
    print("#" * 50)

    # Callbacks
    # mdl_ch = ModelCheckpoint(f"../results/models/CNN_v1_f{i + 1}.h5", monitor="val_class_accuracy",
    #                          save_best_only=True, save_weights_only=True, verbose=0)
    # lr_red = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, min_lr=1e-6)
    # lr_sched = LearningRateScheduler(lrfn)
    # earlystop = EarlyStopping(monitor="val_loss", patience=5)

    # Fit the model
    K.clear_session()
    model = build_model()
    history = model.fit([X_train, np.expand_dims(X_train, axis=-1)], [y_train, y_train_level],
                        batch_size=batch_size,
                        validation_batch_size=2 * batch_size,
                        epochs=epochs,
                        validation_data=([X_val, np.expand_dims(X_val, axis=-1)], [y_val, y_val_level]),
                        callbacks=[], verbose=0)

    val_acc.append(np.max(model.history.history["val_class_accuracy"]))
    val_mae.append(np.min(model.history.history["val_level_mae"]))

    # Inference on test set
    Y_pred = model.predict([fmri_test, np.expand_dims(fmri_test, axis=-1)], verbose=0)
    preds.append(Y_pred[0])
    pred_classes = np.argmax(Y_pred[0], axis=1) - 1
    true_classes = data_test["class"][test_idx].astype(int)
    class_error = 1 - (np.sum(pred_classes[test_idx] == true_classes) / len(true_classes))
    test_count.append(np.sum(pred_classes[test_idx] == true_classes))
    accs.append(accuracy_score(true_classes, pred_classes[test_idx]))
    test_score.append(class_error)

    # Plot the training history
    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 2, 1)
    # plt.plot(history.history["loss"], label="Train Loss")
    # plt.plot(history.history["val_loss"], label="Validation Loss")
    # plt.legend()
    # plt.title("Loss over Epochs")
    # 
    # plt.subplot(1, 2, 2)
    # plt.plot(history.history["accuracy"], label="Train Accuracy")
    # plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    # plt.legend()
    # plt.title("Accuracy over Epochs")
    # plt.show()

    del model
    gc.collect()

print("#" * 100)
print(np.round(val_acc, 2))
print("#" * 50)
print("Acc stats:")
print(np.mean(val_acc), np.std(val_acc), np.min(val_acc), np.max(val_acc))
print("MAE stats:")
print(np.mean(val_mae), np.std(val_mae), np.min(val_mae), np.max(val_mae))
print("#" * 100)
print("Test stats:")
print(f"Correct predictions: {test_count}")
print(f"Scores: {test_score}")


##################################################
### Fold 1
### train size 450, valid size 30
##################################################
##################################################
### Fold 2
### train size 450, valid size 30
##################################################
##################################################
### Fold 3
### train size 450, valid size 30
##################################################
##################################################
### Fold 4
### train size 450, valid size 30
##################################################
##################################################
### Fold 5
### train size 450, valid size 30
##################################################
##################################################
### Fold 6
### train size 450, valid size 30
##################################################
##################################################
### Fold 7
### train size 450, valid size 30
######################

In [1]:
p = np.mean(preds, axis=0)

ConfusionMatrixDisplay.from_predictions(y_test - 1, np.argmax(preds[-1][test_idx], axis=1) - 1, cmap=plt.cm.Blues)
plt.grid(False)
plt.show()

NameError: name 'np' is not defined

In [ ]:
# Phase2 Test labels accuracy
np.mean(accs)